# Recalculer les notes pour les exercices de glose

Avec le comportement *Interactif avec essais multiples*, les questions ne tiennent plus compte des notes pour les réponses partiellement correctes depuis 2019.

Pour contourner le problème, on peut : 
- aller chercher les tentatives des étudiants directement dans le quizz les réponses des participants et exporter les données *fEtudiants* en sélectionnant :
 - toutes les tentatives
 - texte de la question, réponse
 - télécharger au format Excel
- aller exporter les questions du quizz au format Moodle XML *fGloses*

On analyse les questions dans *fGloses* pour produire un dictionnaire :
- la phonétique à gloser (clé)
- la pénalité par essai
- les différents points d'évaluation
 - note
 - expression régulière
 - feedback
- le nom Moodle de la question

On analyse les tentatives dans *fEtudiants* pour faire une DataFrame avec les réponses aux différentes questions.

On calcule les notes et on compare avec l'évaluation stockée par *fEtudiants*. Si elles sont identiques à l'arrondi près, on ne fait rien. Sinon, on affiche l'évaluation de chaque sous-question, la nouvelle note globale et la note stockée pour correction sur eCampus.


In [1]:
import pandas as pd
from lxml import etree as ET
import bs4,re
from cellbell import ding

In [2]:
rep=u"/Users/gilles/Downloads/"

# fEtudiants=rep+u"LNS1U4-LNS1T2-Contrôle continu Découpages simples (Session 2)-réponses.xlsx"
# fGloses=rep+u"quiz-LNS1U4-LNS1T2-2016-20200122-1038.xml"

fEtudiants=rep+u"LNS1U4-LNS1T2-Contrôle continu Découpages simples-réponses.xlsx"
fGloses=rep+u"quiz-LNS1U4-LNS1T2-2016-20210106-0907.xml"
# fGloses=rep+u"quiz-LNS1U4-LNS1T2-2016-20200122-1038.xml"


fEtudiants=rep+u"LNS1U4-LNS1T2-Contrôle continu Découpages en morphes-réponses.xlsx"
fGloses=rep+u"quiz-LNS1U4-LNS1T2-2016-20210106-1136.xml"
# fEtudiants=rep+u"LNS1U4-LNS1T2-Contrôle continu  Découpages en morphes-réponses.xlsx"
# fGloses=rep+u"quiz-LNS1U4-LNS1T2-2016-20200122-1719.xml"

# fEtudiants=rep+u"LNS1U4-LNS1T2-Contrôle continu Découpages en morphes (Session 2)-réponses.xlsx"
# fGloses=rep+u"quiz-LNS1U4-LNS1T2-2016-20200122-1719.xml"


debug=0

In [3]:
gloses=ET.parse(fGloses)
print ET.tostring(gloses)


<quiz>
<!-- question: 0  -->
  <question type="category">
    <category>
      <text>$course$/top/Gloses</text>
    </category>
    <info format="moodle_auto_format">
      <text/>
    </info>
    <idnumber/>
  </question>

<!-- question: 0  -->
  <question type="category">
    <category>
      <text>$course$/top/Gloses/S&#233;paration des morphes</text>
    </category>
    <info format="moodle_auto_format">
      <text/>
    </info>
    <idnumber/>
  </question>

<!-- question: 0  -->
  <question type="category">
    <category>
      <text>$course$/top/Gloses/S&#233;paration des morphes/Simple</text>
    </category>
    <info format="moodle_auto_format">
      <text/>
    </info>
    <idnumber/>
  </question>

<!-- question: 0  -->
  <question type="category">
    <category>
      <text>$course$/top/Gloses/S&#233;paration des morphes/Simple/2016</text>
    </category>
    <info format="moodle_auto_format">
      <text/>
    </info>
    <idnumber/>
  </question>

<!-- question: 836226 

In [4]:
for q in gloses.xpath('//question'):
    for qt in q.findall("questiontext/text"):
        print qt.text
        print
    print "======"

Séparer les mots par des espaces, les morphes par des « - » et les clitiques par des « = » :
<br>amɛnmwaœ̃nuvʁəbwatdəlakɥizin
<br>{1:RXC:=amɛn=mwa œ̃-n uvʁə-bwat də l-a kɥizin#Bravo ! La phrase était : Amène-moi un ouvre-boîte de la cuisine.
<br>~^([^ ]+ +)?([^ ]+ +)?[^ ]+$#Il y a plus de mots
<br>~^([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)*[^ ]+$#Il y a moins de mots
<br>~--.* kɥizin#Il y a un nom facile à identifier.
<br>~%18%--.* uvʁə-bwat .*#Attention au nom composé.
<br>~%28%--.* d-?ə l-?a .*#Attention à la <a href="https://fr.wikipedia.org/wiki/préposition" target="_blank" style="color:blue">préposition</a>.
<br>~%37%--.* də .*#Considérez la <a href="https://fr.wikipedia.org/wiki/préposition" target="_blank" style="color:blue">préposition</a> comme un seul morphe.
<br>~%46%--.* œ̃-n .*#Attention à la <a href="https://fr.wikipedia.org/wiki/Liaison_en_français" target="_blank" style="color:blue">liaison</a> avec l'indéfini.
<br>~%56%--.* l-a 

In [57]:
def analysePercent(lValRep):
    result=1
    if lValRep!="=":
        result=float(lValRep.strip("%"))/100
    return result

def analysePart(lPart):
    valRep=0
    patRep=""
    backRep=""
    m=re.match(ur"(%\d+%|=)?([^#]+)#(.*)",lPart)
    if m:
        if m.group(1)!=None:
            valRep=analysePercent(m.group(1))
        patRep=m.group(2)
        backRep=m.group(3)
    else:
        print "CLOZE : mauvais format"
    return valRep,patRep,backRep

def analyseQuestion(lQuestion):
    questionParts=[]
    phonetique=""
    m=re.search(ur"<br/?>(<span[^>]*>)?([^<>\n]*)(</span>)?\n?<br/?>\{\d+:\w+:([^}]+)\}",lQuestion)
    if m:
        phonetique=m.group(2)
#         print phonetique
        parts=m.group(4).split(u"~")
        for part in parts:
            questionParts.append(analysePart(part))
#     else:
#         print "pas de CLOZE"
    return phonetique,questionParts

In [58]:
def getText(lNode,lSubnode):
    text=""
    for ligne in lNode.findall(u"%s/text"%lSubnode):
        text+=ligne.text
    return text    

def getValue(lNode,lSubnode):
    return float(lNode.find(lSubnode).text)

def getQuestion(node):
    result={}
    tags= [n.tag for n in list(node)]
    if "questiontext" in tags:
        result["question"]=getText(node,"questiontext")
        result["nom"]=getText(node,"name")
        result["penalite"]=getValue(node,"penalty")
        result["essais"]=len(node.findall(u"hint"))
    return result  

questions=[]
for q in gloses.xpath('//question'):
    question=getQuestion(q)
    if question:
        questions.append(question)

exercices={}
for question in questions:
#     print question["nom"]
#     print question["question"]
#     print question["penalite"]
#     print question["essais"]
#     print
    nom, qParts=analyseQuestion(question["question"])
    if nom!="":
        eq=exercices[nom]={}
        eq["penalite"]=question["penalite"]
        eq["essais"]=question["essais"]
        eq["question"]=qParts
        eq["phrase"]=question["nom"]
    else:
        print "=========="
        print question["question"]

for exercice in exercices:
    print exercice
    ee=exercices[exercice]
    print ee["question"]
    print ee["essais"]
    print ee["penalite"]
    print ee["phrase"]
    print

ʒɑ̃nɛpʁipuʁmɔ̃gʁadavɛktebetizpɑ̃dɑ̃levakɑ̃s
[(1, u'\u0292=\u0251\u0303-n=\u025b p\u0281i pu\u0281 m-\u0254\u0303 g\u0281ad av\u025bk t-e betiz p\u0251\u0303d\u0251\u0303 l-e vak\u0251\u0303s', u"Bravo ! La phrase \xe9tait : J'en ai pris pour mon grade avec tes b\xeatises pendant les vacances."), (0, u'^([^ ]+ +)?([^ ]+ +)?([^ ]+ +)?([^ ]+ +)?([^ ]+ +)?([^ ]+ +)?([^ ]+ +)?[^ ]+$', u'Il y a plus de mots'), (0, u'^([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)([^ ]+ +)*[^ ]+$', u'Il y a moins de mots'), (0, u'--.*( p\u0281i .* g\u0281ad .* betiz .* vak\u0251\u0303s).*', u'Il y a au moins quatre mots simples faciles \xe0 identifier : trois noms et un verbe.'), (0.21, u'--.*( pu\u0281 .* av\u025bk .* p\u0251\u0303d\u0251\u0303 ).*', u'Attention aux <a href="https://fr.wikipedia.org/wiki/pr\xe9position" target="_blank" style="color:blue">pr\xe9position</a>s.'), (0.32, u'--.*( m-\u0254\u0303 .* t-e .* l-e ).*', u'Attention

# Analyse des tentatives des étudiants

On regroupe les réponses par le *Nom Prénom* de l'étudiant et *Terminé*.
- les réponses sont listées
- la première formulation des question 
- la dernière valeur de la note

On déploie les tentatives dans des colonnes R1-n et R2-n.

On filtre les questions pour ne garder que la phonétique.

On filtre les réponses pour éliminer l'entête "partie 1 : ".

In [59]:
df=pd.read_excel(fEtudiants)
df.head(1)

,Nom,Prénom,Numéro d'identification,Adresse de courriel,État,Commencé le,Terminé,Temps utilisé,Note/20,Question 1,Réponse 1,Réponse correcte 1,Question 2,Réponse 2,Réponse correcte 2
0,BALLAY,ANTONY,396433,antony.ballay@etu.u-bordeaux-montaigne.fr,NaN,2 décembre 2020 16:08,11 décembre 2020 15:24,8 jours 23 heures,NaN,"Séparer les mots par des espaces, les morphes ...",partie 1 : e-dy solɛj kɑ̃t-ɔ̃=mɛtʁa le vwal,partie 1 : e d-y solɛj kɑ̃-t ɔ̃=mɛt-ʁa l-e vwal,"Séparer les mots par des espaces, les morphes ...",partie 1 : sɛt=ɑ̃fɑ̃-ɛt=aʁive o=lise=ɑ̃ bys,partie 1 : s-ɛt ɑ̃fɑ̃ ɛ-t aʁiv-e o lise ɑ̃ bys


In [60]:
tentatives=df.groupby(u"Nom Prénom Terminé".split(" ")).agg(
    {
    u"Question 1":"first",
    u"Question 2":"first",
    u"Réponse 1":lambda x: [e for e in x],
    u"Réponse 2":lambda x: [e for e in x],
    u"Note/20":"last"
    }).reset_index()

La phonétique de la question se trouve entre le premier et le second "\n". C'est la phonétique qui permet de retrouver la question dans les *exercices*.

In [61]:
tentatives[u"Question 1"]=tentatives[u"Question 1"].str.split("\n").str[1]
tentatives[u"Question 2"]=tentatives[u"Question 2"].str.split("\n").str[1]

On déploie les tentatives en colonnes.

In [62]:
maxTentatives=max(tentatives[u"Réponse 1"].str.len().max(),tentatives[u"Réponse 2"].str.len().max())

listR1=["R1-%d"%(i+1) for i in range(maxTentatives)]
listR2=["R2-%d"%(i+1) for i in range(maxTentatives)]

tentatives[listR1]=pd.DataFrame(tentatives[u"Réponse 1"].values.tolist(), index= tentatives.index)
tentatives[listR2]=pd.DataFrame(tentatives[u"Réponse 2"].values.tolist(), index= tentatives.index)

Fonction pour supprimer l'entête "partie 1 : ".

In [63]:
def formatReponse(valeur):
    result=""
    if valeur and len(valeur)>11:
        result=valeur[11:]
    return result

In [64]:
listCols=u"Nom Prénom Terminé Note/20".split(" ")+[u"Question 1"]+listR1+[u"Question 2"]+listR2
dfTentatives=tentatives[listCols]

In [65]:
for rep in listR1+listR2:
    dfTentatives.loc[:,rep]=dfTentatives.loc[:,rep].apply(formatReponse)

Fonction pour évaluer une réponse à partir des expressions régulières d'une question.

In [66]:
def evaluateRep(lRep,qParts):
    lRep=re.sub(ur"\s+"," ",lRep)
    result=0.85
    for (v,p,fb) in qParts:
        if p.startswith("--"):
            negP=p[2:]
            m=re.match(negP,lRep)
            if not m:
                result=v
                if debug:
                    print lRep
                    print p
                    print fb
                break
        else:
            m=re.match(p,lRep)
            if m:
                result=v
                if debug:
                    print lRep
                    print p
                    print fb
                break
    return result

Fonction pour évaluer une série de réponse à une question en tenant compte des pénalités cumulées.

In [67]:
def evaluateReps(lReps,lParts,lPenalty):
    note=0
    for nRep,rep in enumerate(lReps):
        if r[rep]!="":
            essai=evaluateRep(r[rep],lParts)
            essaiPenalise=essai-lPenalty*nRep
            note=max(essaiPenalise,note)
            if debug:
                print "essai numéro",nRep
                print essai, lPenalty*nRep, "=>", essaiPenalise
                print
        else:
            break
    return note

# Boucle principale

Pour chaque tentative (étudiant, date, note), on recalcule les évaluations pour les deux sous-questions.

Si la note recalculée est la même, à l'arrondi près, que l'ancienne, on ne fait rien.
Sinon, on affiche l'étudiant, la date, la note1, la note2, la note globale et l'ancienne pour correction sur eCampus (avec la phonétique des deux questions)

In [68]:
sortieCorrections=[]
for ix,r in dfTentatives.iterrows():
    qPhonetique=r[u"Question 1"]
    qNoteOriginale=r[u"Note/20"]
    if qNoteOriginale=="-":
        qNoteOriginale=float(0)
    else:
        qNoteOriginale=float(r[u"Note/20"])
    qe=exercices[qPhonetique]
    qParts=qe["question"]
    qPenalty=qe["penalite"]
    note1=evaluateReps(listR1,qParts,qPenalty)

    qPhonetique=r[u"Question 2"]
    qe=exercices[qPhonetique]
    qParts=qe["question"]
    qPenalty=qe["penalite"]
    note2=evaluateReps(listR2,qParts,qPenalty)
    
    noteTentative=float(note1+note2)/2*20
    if round(noteTentative)!=qNoteOriginale:
        sortieCorrections.append((r[u"Nom"],r[u"Prénom"],r[u"Terminé"],note1,note2,noteTentative,qNoteOriginale,r[u"Question 1"],r[u"Question 2"]))
        if debug:
            print r[u"Nom"],r[u"Prénom"],r[u"Terminé"],
            print u"note1 =>",note1,
            print u"note2 =>",note2,
            print u"note => %0.2f"%noteTentative, "(%0.2f)"%qNoteOriginale
            print
    else:
        if debug:
            print "Note identique"
    if debug: 
        print "=============="
        print
print sortieCorrections

[(u'ARCENNATHURY', u'CHARLOTTE', u'16 d\xe9cembre 2020  12:14', 0.0, 0.32, 3.2, 0.0, u'de\u0281izj\u0254\u0303d\u0259nude\u0281izwa\u0281ka\u0281fuls\u0251\u0303tim\u0251\u0303tal\u0254\u0303naswafdideal', u'\u0292\u0251\u0303n\u025bp\u0281ipu\u0281m\u0254\u0303g\u0281adav\u025bktebetizp\u0251\u0303d\u0251\u0303levak\u0251\u0303s'), (u'ARDOUIN', u'HELENE', u'14 d\xe9cembre 2020  22:48', 0.0, 0.8, 8.0, 0.0, u'et\u0281istes\u0251\u0303zok\u0153\u0303nav\u0251\u0303ta\u0292\u0254\u0303nuz\u025b\u0303fli\u0292dedezi\u0281kinuzafli\u0292', u'\u0153\u0303sav\u0251\u0303t\u0251\u0303gl\u025bm\u0259ladi'), (u'AUBIER', u'CLARA', u'15 d\xe9cembre 2020  10:09', 0.28, 0.55, 8.3, 0.0, u'd\u0254nl\u0259l\u0265iav\u0251\u0303l\u0259debyd\u0259lad\xf8zj\u025bmmit\u0251\u0303', u'\u0153\u0303sav\u0251\u0303t\u0251\u0303gl\u025bm\u0259ladi'), (u'AUBIER', u'CLARA', u'15 d\xe9cembre 2020  10:32', 0.12, 0.21, 3.3, 0.0, u's\u0254\u0303nidesog\u0281\u0259nyn\u0259vj\u025b\u0303pad\u0259mwa', u'val\u0259l\u02

In [69]:
pd.DataFrame(sortieCorrections,columns=u"Nom Prénom Terminé NoteQ1 NoteQ2 Note AncienneNote Q1 Q2".split(" ")).to_excel(fEtudiants.replace(".xls","-Recalc.xls"))

In [ ]:
ding()

In [56]:
# print u'ilavym\u0251\u0303kdamu\u0281m\u0251\u0303kda\u0281\u0292\u0251\u0303k\u0254mlavisedet\u025b\u0281\u0292\u0251\u0303ek\u0254msanetwale\u0292\u0251\u0303'
print "\n".join(exercices.keys())
# questions
# dfTentatives

ilatuʁnesavidɑ̃tulesɑ̃spuʁsavwaʁsisaaveœ̃sɑ̃slɛgzistɑ̃s

mɛzavɛksɛtkɔ̃disjɔ̃kilnəpuvɛlɛ̃tɛχpχɛteapɛχsɔnsɑ̃sɛkspozeapɛʁdʁlavi

səkilɑ̃peʃɛdəkomynikeleʃozkilavɛapχizpaʁləmwajɛ̃dəsədɔ̃

ilsədegaʒdəsekaʁtɔ̃dɑ̃balaʒdeʒɑ̃laveɔʁdyzaʒ

ilavɛlədɔ̃dɑ̃tɑ̃dʁləlɑ̃gaʒdebɛt

ilfəzɛnuʁiʁyngʁɑ̃dkɑ̃titedətutsɔχtdəbetaj

etʁistesɑ̃zokœ̃navɑ̃taʒɔ̃nuzɛ̃fliʒdedeziʁkinuzafliʒ

ilavylɛspaskipasɑ̃tʁladʒɛtsɛtlefastlepalasepɥiletɛknisjɛ̃dəsyʁfas

sɛtɑ̃fɑ̃ɛtaʁiveoliseɑ̃bys

sɔ̃nidesogʁənynəvjɛ̃padəmwa

dɑ̃tɔ̃notomobiltuledøɔ̃səʁabjɛ̃

valəlɥidiʁɑ̃fasozjødətuləmɔ̃d

ʒɑ̃nɛpʁipuʁmɔ̃gʁadavɛktebetizpɑ̃dɑ̃levakɑ̃s

ləʁozkɔ̃nupʁopozdavwaʁlekɑ̃titedʃozkidɔnɑ̃vidotʁəʃoz

iladəmɑ̃delœʁaviadetadəʒɑ̃ʁaviʁavidədonelœʁavisyʁlavi
ilavymɑ̃kdamuʁmɑ̃kdaʁʒɑ̃kɔmlavisedetɛʁʒɑ̃ekɔmsanetwaleʒɑ̃
emililamizdɑ̃sɔ̃potoføsəmatɛ̃

edɑ̃ləsjɛliljoʁadezetwal

ɔ̃sɑ̃niʁatutlavidɑ̃seləkalipsoɑ̃nitali

fulsɑ̃timɑ̃talilfovwaʁkɔmɔ̃nupaʁl

memwakɑ̃ʒətjɛ̃tjɛ̃memwakɑ̃ʒətjɛ̃ladɑ̃medømɛ̃zeblui
kamijɑ̃namipaʁtusyʁtezafɛʁ

ɔ̃nupʁɑ̃fopadekonedekɔ̃nenepuʁdekɔ